In [ ]:
import requests
import pandas as pd 
import folium
from folium import FeatureGroup, features, CircleMarker
import numpy as np
from matplotlib import colors as mcolors
from haversine import haversine
import traceback
from folium import IFrame
from IPython.display import Markdown, display

In [ ]:
query=\
"""
[out:json];
way[highway]
  [highway!~"(pedestrian|path|footway|steps|via_ferrata|service|construction|unclassified)"]
  [tunnel][bicycle=no]
  (45.82114340079471,5.7952880859375,47.73562905149295,12.1871337890625);
(._;>;);
out;"""

In [ ]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query)
print(r.status_code)
ores = r.json()

In [ ]:
tunnels =[e for e in ores['elements'] if e['type']=='way']
len(tunnels)

In [ ]:
nodes = [e for e in ores['elements'] if e['type']=='node']

In [ ]:
tunnelnodes = []
for tunnel in tunnels:
    nt = []
    for nodeid in tunnel['nodes']:
        for node in nodes:
            if node['id'] == nodeid:
                nt.append([node['lat'],node['lon']])
    tunnelnodes.append(nt)

# All forbidden tunnels in the Alps

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i,tunnel in enumerate(tunnels):
    folium.PolyLine(tunnelnodes[i],color='red').add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
mypath='./2019-08-05/Tracks/'

In [ ]:
racers = pd.read_pickle('racers.pkl')

In [ ]:
route={}
for racer in racers['Name']:
    try:
        ll =[l[0] for i,l in enumerate(pd.read_pickle(mypath+racer+'.pkl').values.tolist())]
        route[racer] = ll
    except Exception as e:
        print('error: '+racer+' '+traceback.format_exc())

In [ ]:
mycolors = list(mcolors.cnames.keys())

In [ ]:
tunnellength = [sum(list(map(lambda x,y:haversine(x,y),tunnel[1:],tunnel[:-2]))) for tunnel in tunnelnodes]

In [ ]:
bg =[]
for k,racer in racers.iterrows():
    try:
        if racer['Bib']=='CC1' or racer['Bib']=='CC2':
            continue
        for i,l in enumerate(route[racer['Name']]):
            for j,tunnel in enumerate(tunnelnodes):
                d = haversine(l,tunnel[0],'km')
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
                d = haversine(l,tunnel[-1],'km')                    
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
    except:
        print(racer['Name'])

## Forbidden tunnels where riders came close

In [ ]:
bgt =list(set([e[0] for e in bg]))
len(bgt)

In [ ]:
def popup(t,show=False):
    return folium.Popup('%s' % (t),show=show)

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i in bgt:
    name = name = tunnels[i]['tags']['name'] if 'name' in tunnels[i]['tags'] else ''
    folium.PolyLine(tunnelnodes[i],color='red',popup=popup(name)).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[0]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

Josh Murphy looks suspicious

In [ ]:
name = 'Josh_Murphy'
ll = route[name]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[1],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[1][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[1][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
m.fit_bounds(m.get_bounds())
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m

In [ ]:
ibgt = bgt[1]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
name = 'Rados__aw_Rog____'
pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
ll = route[name][pt[0]-1:pt[-1]+1]
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
[CircleMarker(l,popup=popup(name)).add_to(m) for l in ll]
folium.PolyLine(ll).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[2]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))


In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[3]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[4]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[5]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
    #folium.PolyLine(ll,color=mycolors[i],tooltip=name,highlight_function=highlight_function).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[6]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[7]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[8]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[9]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[10]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[11]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[11]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[12]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt

In [ ]:
ibgt = bgt[13]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m

In [ ]:
ibgt = bgt[14]
display(Markdown('# '+tunnels[ibgt]['tags']['name']))

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[ibgt],color='red',popup=popup('Tunnel')).add_to(m)
CircleMarker(tunnelnodes[ibgt][0],radius=2,color='purple',fillcolor='purple').add_to(m)
CircleMarker(tunnelnodes[ibgt][-1],radius=2,color='purple',fillcolor='purple').add_to(m)
names = list(set([e[4] for e in bg if e[0]==ibgt and e[0]==ibgt]))
for i,name in enumerate(names):
    pt = sorted([e[3] for e in bg if e[0]==ibgt and e[4]==name])
    ll = route[name][pt[0]:pt[-1]]
    [CircleMarker(l,color=mycolors[i],radius=2,popup=popup(name)).add_to(m) for l in ll]
m.fit_bounds(m.get_bounds())
m